# Classification

 This notebook assumes that models directory is located in the same directory as this notebook.  
(i.e. Each model should be saved under /../../saved_models/)  
This notebook assumes that image directory is located unde ./classification/samples/images/  

Import packages

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from IPython.display import Image
import matplotlib.pyplot as plt
import ipywidgets as widgets
import itertools
import os
import shutil
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

Model and Dataset selection

In [ ]:
models_PATH = '../saved_models/'
list_of_models = [dir for dir in os.listdir(models_PATH) if not dir.startswith('.')]
dropdown_model = widgets.Dropdown(
    options=list_of_models,
    description='Model',
)
display(dropdown_model)

## Keras Model

Image size selection

In [ ]:
size = widgets.IntText(
    description='Image Dim.:',
    disabled=False
)
display(size)

Set up parameters

In [ ]:
batch_size=1
sample_path = '../classification/samples/'
IMG_HEIGHT = size.value
IMG_WIDTH = size.value

Format data, load images and apply rescaling

In [ ]:
sample_image_gen = ImageDataGenerator(rescale=1. / 255).flow_from_directory(batch_size=batch_size,
                                                                            directory=sample_path,
                                                                            target_size=(IMG_HEIGHT, IMG_WIDTH))
                                                                        

Load model

In [ ]:
model_PATH = os.path.join(models_PATH, dropdown_model.value + '/my_model.h5')

model = tf.keras.models.load_model(model_PATH)
    
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

Predict

In [ ]:
predictions_probabilities = model.predict(sample_image_gen, batch_size=None, verbose=1)

## Classify

Seperate results amongst classes

In [ ]:
files = []
for file_dir in sample_image_gen.filepaths:
    files.append(os.path.split(file_dir)[1])

results = zip(predictions_probabilities, predictions_probabilities.argmax(axis=-1), files, list(sample_image_gen.filepaths))
fakes, reals = [], []

for element in results:
    if element[1] == 0:
        fakes.append(element)
    else:
        reals.append(element)
        
print(fakes[0][3])

Move files to respective directories

In [ ]:
fake_dir = '../classification/results/fake/'
real_dir = '../classification/results/real/'


for element in fakes:
    shutil.move(element[3], fake_dir)

for element in reals:
    shutil.move(element[3], real_dir)

Display most probable fake images (Top 10)

In [ ]:
# Sort based on probabilities
fakes.sort(key=lambda y: y[0][0], reverse=True)
    
for i in range(len(fakes)):
    if i == 10:
        break
    display(Image(filename=os.path.join(fake_dir, fakes[i][2])))